In [1]:
!pip install mediapipe opencv-python numpy

  Using cached mediapipe-0.10.21-cp312-cp312-win_amd64.whl.metadata (10 kB)
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached jax-0.6.1-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.6.1-cp312-cp312-win_amd64.whl.metadata (1.2 kB)
  Using cached matplotlib-3.10.3-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached protobuf-4.25.7-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached sounddevice-0.5.2-py3-none-win_amd64.whl.metadata (1.6 kB)
  Using cached sentencepiece-0.2.0-cp312-cp312-win_amd

In [ ]:
import mediapipe as mp
import cv2
import numpy as np
import socket

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

serverAddressPort = ('127.0.0.1', 12345)
#sock.connect(serverAddressPort)

with mp_hands.Hands() as hands:

    while cap.isOpened():

        ret, frame = cap.read()

        if not ret:
            print("ignoring empty frame")
            continue

        image = frame
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image_height, image_width, _ = image.shape

        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand)
                print(hand)
            for name in results.multi_hand_world_landmarks.landmark:
                for i, val in enumerate(name):
                    print(val)
            # for i, hand in enumerate(results.multi_hand_world_landmarks):
            #     print(hand[0])
            
            # sock.send(str.encode(str(results.multi_hand_world_landmarks)))

        cv2.imshow("hand tracking", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            sock.close()
            break
cap.release()
cv2.destroyAllWindows()

landmark {
  x: 0.428536057
  y: 0.839953125
  z: 1.12905013e-006
}
landmark {
  x: 0.42632854
  y: 0.829063118
  z: -0.09117046
}
landmark {
  x: 0.399322331
  y: 0.820455194
  z: -0.164188862
}
landmark {
  x: 0.423628688
  y: 0.789652109
  z: -0.232199907
}
landmark {
  x: 0.468459845
  y: 0.769234717
  z: -0.302956402
}
landmark {
  x: 0.0896811783
  y: 0.746404171
  z: -0.0936158821
}
landmark {
  x: -0.0360695273
  y: 0.694258
  z: -0.146431878
}
landmark {
  x: -0.112066522
  y: 0.672049463
  z: -0.192813739
}
landmark {
  x: -0.171322703
  y: 0.654741049
  z: -0.228620067
}
landmark {
  x: 0.0700439215
  y: 0.715140522
  z: -0.0878951475
}
landmark {
  x: -0.0648255348
  y: 0.669726491
  z: -0.136634529
}
landmark {
  x: -0.139214575
  y: 0.650904596
  z: -0.177718714
}
landmark {
  x: -0.193121895
  y: 0.638969
  z: -0.207924649
}
landmark {
  x: 0.0808456093
  y: 0.684197903
  z: -0.0907727107
}
landmark {
  x: -0.0496998429
  y: 0.636648834
  z: -0.132438898
}
landmark {
  x

AttributeError: 'list' object has no attribute 'landmark'

: 

In [ ]:
#visualizer
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image


# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import numpy as np
import socket

# STEP 2: Create an HandLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2, min_hand_detection_confidence = 0.5, min_hand_presence_confidence = 0.5, min_tracking_confidence = 0.5, running_mode=vision.RunningMode.VIDEO)
detector = vision.HandLandmarker.create_from_options(options)

cap = cv2.VideoCapture(0)

mp_drawing = mp.solutions.drawing_utils

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

serverAddressPort = ('127.0.0.1', 12345)
sock.connect(serverAddressPort)

while cap.isOpened():
    # frame = frame + 1
    ret, image = cap.read()

    if not ret:
        print("ignoring empty frame")
        continue
    image.flags.writeable = False
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(mp.ImageFormat.SRGB, image)
    # print(frame * 1000 / frameRate)
    # print(int(cap.get(cv2.CAP_PROP_POS_MSEC)))
    # results = detector.detect_for_video(mp_image, int(frame * 1000 / frameRate))
    results = detector.detect_for_video(mp_image, int(cap.get(cv2.CAP_PROP_POS_MSEC)))

    # image.flags.writeable = True
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image_height, image_width, _ = image.shape

    if results.hand_world_landmarks:
        for num, hand in enumerate(results.hand_landmarks):
            image = draw_landmarks_on_image(image, results)
            for point, landmark in enumerate(hand):
                sock.send(str.encode("{%02d %.2f %.2f %.2f}" % (point, landmark.x, landmark.y, landmark.z)))
                print("%02d %.2f %.2f %.2f" % (point, landmark.x, landmark.y, landmark.z))
        

    cv2.imshow("hand tracking", image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        sock.close()
        break
cap.release()
cv2.destroyAllWindows()
# STEP 3: Load the input image.
# image = mp.Image.create_from_file("image.jpg")

# # STEP 4: Detect hand landmarks from the input image.
# detection_result = detector.detect(image)

# # STEP 5: Process the classification result. In this case, visualize it.
# annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
# cv2_imshow(cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))

00 0.55 0.90 0.00
01 0.58 0.76 -0.04
02 0.57 0.63 -0.04
03 0.54 0.52 -0.04
04 0.50 0.44 -0.05
05 0.45 0.60 -0.01
06 0.39 0.49 -0.03
07 0.35 0.43 -0.05
08 0.31 0.38 -0.06
09 0.41 0.65 -0.01
10 0.32 0.57 -0.03
11 0.25 0.52 -0.06
12 0.20 0.48 -0.08
13 0.38 0.71 -0.01
14 0.29 0.65 -0.04
15 0.24 0.61 -0.07
16 0.19 0.58 -0.09
17 0.38 0.79 -0.02
18 0.30 0.75 -0.04
19 0.25 0.73 -0.06
20 0.21 0.71 -0.08
00 0.55 0.85 0.00
01 0.56 0.73 -0.04
02 0.55 0.60 -0.06
03 0.51 0.51 -0.07
04 0.48 0.44 -0.08
05 0.43 0.58 -0.03
06 0.37 0.48 -0.06
07 0.33 0.43 -0.08
08 0.29 0.38 -0.10
09 0.39 0.64 -0.03
10 0.30 0.56 -0.06
11 0.25 0.52 -0.08
12 0.20 0.48 -0.10
13 0.37 0.70 -0.04
14 0.28 0.64 -0.06
15 0.23 0.61 -0.08
16 0.18 0.58 -0.09
17 0.36 0.77 -0.04
18 0.29 0.75 -0.07
19 0.25 0.73 -0.08
20 0.21 0.71 -0.09
00 0.53 0.83 0.00
01 0.54 0.71 -0.04
02 0.53 0.60 -0.05
03 0.49 0.51 -0.06
04 0.46 0.44 -0.08
05 0.41 0.58 -0.02
06 0.35 0.48 -0.05
07 0.31 0.43 -0.07
08 0.27 0.39 -0.08
09 0.37 0.63 -0.02
10 0.29 0.56 -0